# ChipChat Tutorial – Part III Extension
## Parameterized Binary-to-BCD Converter

This extension modifies the tutorial binary-to-BCD converter by adding
a parameter that controls the input width.

The design remains purely combinational and synthesizable.
A custom self-checking testbench is provided to verify correctness.

In [34]:
!apt-get update
!apt-get install -y iverilog

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,755 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iverilog is already the newest version (11.0-1.1).

## Parameterized Binary-to-BCD Converter

- INPUT_WIDTH parameter (default = 5)
- Supports values up to 99
- Packed BCD output

In [35]:
verilog_code = r"""
`timescale 1ns / 1ps

module binary_to_bcd_param #(
    parameter INPUT_WIDTH = 5
)(
    input  wire [INPUT_WIDTH-1:0] binary,
    output reg  [7:0] bcd
);

    always @(*) begin
        bcd[3:0] = binary % 10;
        bcd[7:4] = binary / 10;
    end

endmodule
"""
with open("binary_to_bcd_param.v", "w") as f:
    f.write(verilog_code)

## Self-Checking Testbench

This testbench:
- Tests all values from 0 to 31
- Automatically checks BCD correctness

In [36]:
tb_code = r"""
`timescale 1ns / 1ps

module tb_binary_to_bcd_param;

    reg  [4:0] binary;
    wire [7:0] bcd;

    integer i;
    reg [7:0] expected;

    binary_to_bcd_param #(.INPUT_WIDTH(5)) dut (
        .binary(binary),
        .bcd(bcd)
    );

    initial begin
        $display("Starting parameterized BCD test...");

        for (i = 0; i < 32; i = i + 1) begin
            binary = i;
            expected[3:0] = i % 10;
            expected[7:4] = i / 10;
            #1;

            if (bcd !== expected) begin
                $display("ERROR: %0d -> expected %b, got %b",
                         i, expected, bcd);
                $finish;
            end
        end

        $display("All extension tests passed.");
        $finish;
    end

endmodule
"""
with open("binary_to_bcd_param_tb.v", "w") as f:
    f.write(tb_code)

In [38]:
!iverilog -g2012 binary_to_bcd_param.v binary_to_bcd_param_tb.v
!vvp a.out

Starting parameterized BCD test...
All extension tests passed.


We extended the tutorial binary-to-BCD converter by parameterizing the input width.
This allows the same RTL to support different binary sizes while preserving combinational behavior.
A self-checking testbench verifies correctness for all input values.
This demonstrates parameterization and reproducible verification.